In [1]:
import sys
import os
import pandas as pd
import numpy as np
import logging
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import IPython
from typing import List, Tuple, Dict, Any
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import RobustScaler

NOTEBOOK_PATH: Path = Path(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"])
PROJECT_DIR: Path = NOTEBOOK_PATH.parent.parent
sys.path.append(str(PROJECT_DIR))
import src.utils.custom_log as custom_log
from src.utils.set_rcparams import set_rcparams
from src.utils.PathChecker import PathChecker
from src._StandardNames import StandardNames
from src.utils.ParquetHandler import ParquetHandler

os.chdir(PROJECT_DIR)
LOG: logging.Logger = logging.getLogger(__name__)

custom_log.init_logger(logging.INFO)
LOG.info("Log initialized")

set_rcparams()

2024-08-07 16:29:24,578     INFO  MainProcess  MainThread Log initialized
2024-08-07 16:29:24,579     INFO  MainProcess  MainThread Setting rcparams for matplotlib
2024-08-07 16:29:24,611     INFO  MainProcess  MainThread Using style src/visualization/dissertation.mplstyle


In [2]:
STR: StandardNames = StandardNames()

DATA_DIR: Path = PathChecker().check_directory(PROJECT_DIR / "data" / "doe" / "doe_sobol_20240705_194200")

FIGURE_DIR = PROJECT_DIR / "reports" / "figures" / NOTEBOOK_PATH.stem
FIGURE_DIR.mkdir(parents=True, exist_ok=True)
FIGURE_DIR: Path = PathChecker().check_directory(FIGURE_DIR)

DB_FILE: ParquetHandler = ParquetHandler(DATA_DIR / "injury_criteria.parquet")

PERCENTILE: int = 5

CHANNELS: List[str] = [
    "Head_HIC15",
    "Head_HIC36",
    "Head_a3ms",
    "Neck_Nij",
    "Neck_Fz_Max_Compression",
    "Neck_Fz_Max_Tension",
    "Neck_My_Max",
    "Neck_Fx_Shear_Max",
    "Chest_Deflection",
    "Chest_a3ms",
    "Femur_Fz_Max_Compression",
    "Femur_Fz_Max_Tension",
    "Femur_Fz_Max",
]

2024-08-07 16:29:24,621     INFO  MainProcess  MainThread Directory '/root/py_projects/aihiii/data/doe/doe_sobol_20240705_194200' exists
2024-08-07 16:29:24,623     INFO  MainProcess  MainThread Directory '/root/py_projects/aihiii/reports/figures/another_insight_trial_injury_vals' exists
2024-08-07 16:29:24,623     INFO  MainProcess  MainThread File '/root/py_projects/aihiii/data/doe/doe_sobol_20240705_194200/injury_criteria.parquet' exists


In [3]:
def analyse():
    db = DB_FILE.read(columns=CHANNELS, percentiles=[PERCENTILE])
    db_median = pd.DataFrame(np.ones(db.shape) * db.median().to_numpy().reshape(1, -1), index=db.index, columns=db.columns)

    display(db.median())

    scaler = RobustScaler()
    db_scaled = db.copy()
    db_scaled.loc[:, :] = scaler.fit_transform(db)
    db_median_scaled = pd.DataFrame(
        np.ones(db.shape) * db_scaled.median().to_numpy().reshape(1, -1), index=db.index, columns=db.columns
    )

    results = pd.DataFrame(
        {
            "r2_raw": r2_score(db, db_median, multioutput="raw_values"),
            "mae_raw": mean_absolute_error(db, db_median, multioutput="raw_values"),
            "mse_raw": mean_squared_error(db, db_median, multioutput="raw_values"),
            "mape_raw": mean_absolute_percentage_error(db, db_median, multioutput="raw_values"),
            "r2_scaled": r2_score(db_scaled, db_median_scaled, multioutput="raw_values"),
            "mae_scaled": mean_absolute_error(db_scaled, db_median_scaled, multioutput="raw_values"),
            "mse_scaled": mean_squared_error(db_scaled, db_median_scaled, multioutput="raw_values"),
            "mape_scaled": mean_absolute_percentage_error(db_scaled, db_median_scaled, multioutput="raw_values"),

        },
        index=db.columns,
    ).T
    results["Mean"] = results.mean(axis=1)
    results["Sum"] = results.sum(axis=1)

    display(results)

    contributions = results / results["Sum"].values.reshape(-1,1)
    contributions = contributions.drop(columns=["Sum", "Mean"])
    display(contributions.T.describe())


analyse()

Head_HIC15                  258.872803
Head_HIC36                  426.494263
Head_a3ms                    55.214901
Neck_Nij                      0.585954
Neck_Fz_Max_Compression       0.011237
Neck_Fz_Max_Tension           1.078588
Neck_My_Max                  40.165554
Neck_Fx_Shear_Max             0.803787
Chest_Deflection             22.644989
Chest_a3ms                   55.458542
Femur_Fz_Max_Compression      0.521530
Femur_Fz_Max_Tension          1.938342
Femur_Fz_Max                  1.939500
dtype: float32

,Head_HIC15,Head_HIC36,Head_a3ms,Neck_Nij,Neck_Fz_Max_Compression,Neck_Fz_Max_Tension,Neck_My_Max,Neck_Fx_Shear_Max,Chest_Deflection,Chest_a3ms,Femur_Fz_Max_Compression,Femur_Fz_Max_Tension,Femur_Fz_Max,Mean,Sum
r2_raw,-0.041638,-0.003912,-0.001683,-0.005275,-0.136509,-0.048803,-0.000389,-0.008144,-0.045555,-0.000927,-0.002955,-0.002684,-0.003151,-0.023202,-0.324828
mae_raw,102.102069,127.827623,7.911287,0.067870,0.046286,0.209948,5.799469,0.116826,5.979123,4.672606,0.115421,0.137922,0.145749,19.625554,274.757753
mse_raw,16761.034429,23253.167173,90.812364,0.007676,0.013406,0.081837,53.969700,0.024976,52.383933,31.220440,0.109821,0.052246,0.114527,3095.614810,43338.607338
mape_raw,0.397541,0.342315,0.146728,0.116473,1.031981,0.178518,0.152647,0.143678,0.390975,0.084894,0.210422,0.069446,0.070506,0.256625,3.592751
r2_scaled,-0.041638,-0.003912,-0.001683,-0.005275,-0.136509,-0.048803,-0.000389,-0.008144,-0.045555,-0.000927,-0.002955,-0.002684,-0.003151,-0.023202,-0.324828
mae_scaled,0.556071,0.531789,0.537393,0.596391,0.757232,0.613441,0.589751,0.640347,0.472049,0.535639,0.609883,0.519772,0.548486,0.577557,8.085798
mse_scaled,0.497155,0.402450,0.419020,0.592743,3.588033,0.698668,0.558098,0.750375,0.326511,0.410266,3.066220,0.742008,1.621916,1.051805,14.725267
mape_scaled,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,14.000000


,r2_raw,mae_raw,mse_raw,mape_raw,r2_scaled,mae_scaled,mse_scaled,mape_scaled
count,13.000000,13.000000,1.300000e+01,13.000000,13.000000,13.000000,13.000000,1.300000e+01
mean,0.071429,0.071429,7.142857e-02,0.071429,0.071429,0.071429,0.071429,7.142857e-02
std,0.119120,0.155523,1.758635e-01,0.072275,0.119120,0.008767,0.072426,6.457412e-09
min,0.001197,0.000168,1.771255e-07,0.019330,0.001197,0.058380,0.022173,7.142855e-02
25%,0.008264,0.000425,1.205523e-06,0.032419,0.008264,0.066244,0.028456,7.142857e-02
50%,0.012043,0.000764,2.642611e-06,0.042488,0.012043,0.068771,0.040253,7.142857e-02
75%,0.128184,0.021761,1.245303e-03,0.095279,0.128184,0.075426,0.050958,7.142857e-02
max,0.420250,0.465238,5.365462e-01,0.287240,0.420251,0.093650,0.243665,7.142858e-02
